In [21]:
import os
import re
import numpy as np
import pandas as pd
import nltk
from collections import Counter
from scipy.cluster.hierarchy import dendrogram
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
# Modules from scikit-learn
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

In [6]:
fake = pd.read_csv("Dataset/Fake.csv")
true = pd.read_csv("Dataset/True.csv")

In [7]:
## Looping through and adding every text to a string for True
trueCorpus = []
fakeCorpus = []
for i in true['text']:
    trueCorpus.append(i) 

for i in fake['text']:
    fakeCorpus.append(i) 

In [8]:
## Tokenizing texts with NLTK
tokenized_True = [word_tokenize(i) for i in trueCorpus]
tokenized_True = [word_tokenize(i) for i in fakeCorpus]

In [ ]:
print("trueCorpus Length: "+ str(len(tokenized_True)))
print("fakeCorpus Length: "+ str(len(tokenized_Fake)))

In [9]:
## Removing Stopwords from the Corpus
stop_words = set(stopwords.words('english'))
filteredTrue = []
filteredFake = []

for wordList in tokenized_True:
    for word in wordList:
        if word not in stop_words:
            filteredTrue.append(word)

for word in tokenized_True:
    for word in wordList:
        if word not in stop_words:
            filteredFake.append(word)

In [15]:
print(len(filteredTrue))

6935100


In [16]:
print(filteredTrue[0:1000])

['Donald', 'Trump', 'wish', 'Americans', 'Happy', 'New', 'Year', 'leave', '.', 'Instead', ',', 'give', 'shout', 'enemies', ',', 'haters', 'dishonest', 'fake', 'news', 'media', '.', 'The', 'former', 'reality', 'show', 'star', 'one', 'job', '.', 'As', 'Country', 'rapidly', 'grows', 'stronger', 'smarter', ',', 'I', 'want', 'wish', 'friends', ',', 'supporters', ',', 'enemies', ',', 'haters', ',', 'even', 'dishonest', 'Fake', 'News', 'Media', ',', 'Happy', 'Healthy', 'New', 'Year', ',', 'President', 'Angry', 'Pants', 'tweeted', '.', '2018', 'great', 'year', 'America', '!', 'As', 'Country', 'rapidly', 'grows', 'stronger', 'smarter', ',', 'I', 'want', 'wish', 'friends', ',', 'supporters', ',', 'enemies', ',', 'haters', ',', 'even', 'dishonest', 'Fake', 'News', 'Media', ',', 'Happy', 'Healthy', 'New', 'Year', '.', '2018', 'great', 'year', 'America', '!', 'Donald', 'J.', 'Trump', '(', '@', 'realDonaldTrump', ')', 'December', '31', ',', '2017Trump', 'tweet', 'went', 'welll', 'expect.What', 'kind

In [17]:
print(stop_words)

{'she', 'those', 'he', 'shouldn', 'her', 've', 'its', 'during', 'themselves', 'hers', 'then', "you're", 'i', 'couldn', 'who', 'do', 'own', "should've", "you'll", "hasn't", "you'd", 'not', 'this', 'other', 'into', 'had', 'until', 'any', 'm', 'needn', 'through', 'didn', 'them', 'his', 'by', 'against', "wasn't", "didn't", 'before', 're', 'our', 'all', 'nor', 'both', 'having', 'mustn', 'why', 'with', 'isn', 'to', 'below', "doesn't", 'such', 'it', 'wouldn', 'my', 'we', 'or', 'most', 'weren', 'some', 'aren', 'out', 'don', 'your', 'be', 'being', "don't", 'after', 'than', 'they', 'while', 'no', 'very', 'herself', 'over', 'as', 'mightn', 'yours', 'is', "won't", 'did', 'how', 'doesn', 'me', 'if', 'a', 'further', 'wasn', 'at', 'd', 'o', "mightn't", 'above', 'each', 'up', 'again', "couldn't", 'just', 'few', 'where', 'too', 'these', 'were', 'when', "aren't", 'the', 'but', 'was', 'for', 'yourselves', 'of', 'ours', 'between', 'so', 'more', 'been', 'under', "wouldn't", 'll', 'whom', "you've", "isn't",

In [ ]:
##Removing Punctuation 

tempTrue  = ' '.join([str(elem) for elem in filteredTrue])
tokenizer = nltk.RegexpTokenizer(r"\w+")
filteredTrue = tokenizer.tokenize(tempTrue)


tempFalse  = ' '.join([str(elem) for elem in filteredFake])
tokenizer = nltk.RegexpTokenizer(r"\w+")
filteredFake = tokenizer.tokenize(tempFalse)

In [ ]:
len(filteredFake)
len(filteredTrue)